# DSC 40B: Graph Traversal Algorithms
Daniel Lee & Udaikaran Singh

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import graphHelper as gH
import collections
import ipywidgets as widgets

## Graph 

### Undirected Graph Implementation

In [2]:
class Graph:
    """A simple graph data structure."""

    def __init__(self):
        self.adj = dict()

    def add_node(self, label):
        self.adj[label] = set()

    def add_edge(self, u_label, v_label):
        for x in {u_label, v_label}:
            if x not in self.adj:
                self.adj[x] = set()

        self.adj[u_label].add(v_label)
        self.adj[v_label].add(u_label)

    def nodes(self):
        yield from self.adj.keys()

    def edges(self):
        seen = set()

        for u, neighbors in self.adj.items():
            for v in neighbors:
                edge = frozenset((u, v))
                if edge not in seen:
                    seen.add(edge)
                    yield (u, v)
                else:
                    continue

    def neighbors(self, u_label):
        for v in self.adj[u_label]:
            yield v

    def has_node(self, label):
        return label in self.adj

    def has_edge(self, u_label, v_label):
        return v_label in self.adj[u_label]


### Directed Graph Implementation

In [3]:
class DiGraph:
    """A simple directed graph data structure."""

    def __init__(self):
        self.adj = dict()

    def add_node(self, label):
        self.adj[label] = set()

    def add_edge(self, u_label, v_label):
        for x in {u_label, v_label}:
            if x not in self.adj:
                self.adj[x] = set()

        self.adj[u_label].add(v_label)

    def nodes(self):
        yield from self.adj.keys()

    def edges(self):
        for u, neighbors in self.adj.items():
            for v in neighbors:
                edge = frozenset((u, v))
                yield (u, v)

    def neighbors(self, u_label):
        for v in self.adj[u_label]:
            yield v

    def has_node(self, label):
        return label in self.adj

    def has_edge(self, u_label, v_label):
        return v_label in self.adj[u_label]


## Flight Graph

In [4]:
gH.plotMap()

A graph is simply a set of nodes of edges used to represent a system. 

For example, we can use a graph to flight paths. Imagine if all flights were repesented by the graph above. We can use BFS and DFS to:
    - 1. Find out if it is possible to get from node A to node B
    - 2. Find a route from node A to node B
    
note 1: A graph does not have to lie on a coordinate grid

note 2: you can find how the map above was implemented within the graphHelper class

## DFS

In [5]:
Origin = "San Jose"
Destination = "New York"

In [6]:
def dfs(graph, origin, destination):
    parents = {el:None for el in graph.nodes()}
    
    #initial stack
    stack = []

    #add origin to the stack
    stack.append(origin)
    discovered = []
    while (len(stack) != 0):
        curNode = stack.pop()
        if curNode == destination:
            break
        
        if (curNode not in discovered):
            discovered.append(curNode)
            for neighbor in graph.neighbors(curNode):
                if neighbor not in discovered:
                    parents[neighbor] = curNode
                    stack.append(neighbor)

    return parents

## BFS

note: the only substantial difference between how DFS and BFS are implemented if that we use a Queue rather than a Stack

In [7]:
def bfs(graph, origin, destination):
    parents = {el:None for el in graph.nodes()}
    
    #initial queue
    queue = collections.deque()

    #add origin to the queue
    queue.append(origin)
    discovered = []
    while (len(queue) != 0):
        curNode = queue.popleft()
        if curNode == destination:
            break
        if (curNode not in discovered):
            discovered.append(curNode)
            for neighbor in graph.neighbors(curNode):
                if neighbor not in discovered:
                    parents[neighbor] = curNode
                    queue.append(neighbor)

    return parents

### Getting a route

In [8]:
def getRoute(parents, destination):
    route = [destination]
    
    #find the parent node of the destination
    curNode = parents[destination]
    
    while (curNode != None):
        #prepend the parent
        route = [curNode] + route
        curNode = parents[curNode]
    
    return route

### Example 1: From San Jose to Portland

In [9]:
origin = "San Jose"
destination = "Portland"

In [10]:
gH.plotBFS(origin, destination)

Node Searched in each Iteration
Iteration 1: San Jose
Iteration 2: Denver
Iteration 3: Eldrige
Iteration 4: La Jolla
Iteration 5: New Orleans
Iteration 6: Portland


In [11]:
gH.plotDFS(origin, destination)

Node Searched in each Iteration
Iteration 1: San Jose
Iteration 2: New Orleans
Iteration 3: La Jolla
Iteration 4: Chicago
Iteration 5: Seattle
Iteration 6: Louisville
Iteration 7: Orlando
Iteration 8: Kansas City
Iteration 9: Salt Lake City
Iteration 10: Milwaukee
Iteration 11: Houston
Iteration 12: Los Angeles
Iteration 13: Philadelphia
Iteration 14: Nashville
Iteration 15: Toronto
Iteration 16: Atlanta
Iteration 17: Boston
Iteration 18: Boise
Iteration 19: Portland


### Example 2: San Jose to La Jolla

In [12]:
origin = "San Jose"
destination = "La Jolla"

In [13]:
gH.plotBFS(origin, destination)

Node Searched in each Iteration
Iteration 1: San Jose
Iteration 2: Denver
Iteration 3: Eldrige
Iteration 4: La Jolla


In [14]:
gH.plotDFS(origin, destination)

Node Searched in each Iteration
Iteration 1: San Jose
Iteration 2: New Orleans
Iteration 3: La Jolla


## Create your Own Routes

In [38]:
origin = widgets.Dropdown(
    options= list(gH.undirectedGraph.nodes()),
    value='La Jolla',
    description='First:',
)
destination = widgets.Dropdown(
    options=list(gH.undirectedGraph.nodes()),
    value='Toronto',
    description='Second:',
)
traversalMethods = widgets.Dropdown(
    options=["BFS", "DFS"],
    value='DFS',
    description='Traversal:',
)

display(origin)
display(destination)
display(traversalMethods)

Dropdown(description='First:', index=1, options=('San Jose', 'La Jolla', 'Los Angeles', 'Boston', 'Phoenix', '…

Dropdown(description='Second:', index=17, options=('San Jose', 'La Jolla', 'Los Angeles', 'Boston', 'Phoenix',…

Dropdown(description='Traversal:', index=1, options=('BFS', 'DFS'), value='DFS')

Place in a pair of cities and pick your traversal algorithm.

Then run the cell below to see the route that is created

In [42]:
if traversalMethods.value == "DFS":
    map = gH.plotDFS(origin.value, destination.value)
else:
    map = gH.plotBFS(origin.value, destination.value)
map

Node Searched in each Iteration
Iteration 1: La Jolla
Iteration 2: Chicago
Iteration 3: San Jose
Iteration 4: Boston
Iteration 5: Seattle
Iteration 6: Denver
Iteration 7: Eldrige
Iteration 8: New Orleans
Iteration 9: Milwaukee
Iteration 10: Boise
Iteration 11: Philadelphia
Iteration 12: Toronto


## Practice Problems

### Problem 1: Undirected Graph

![Example Graph](https://www.codediesel.com/wp-content/uploads/2012/02/d-graph2.gif)

Encode the above undirected graph. Then, find the shortest route between nodes A and D.
 

In [15]:
getRoute = gH.getRoute

In [16]:
grph = Graph()

In [17]:
grph.add_node("A")
grph.add_node("B")
grph.add_edge("A", "B")
#add your code here to encode the rest of the graph

In [18]:
origin = "A"
destination = #your code here

In [19]:
parents = bfs(graph = grph,
              origin = origin,
              destination = destination)

getRoute(parents, destination)

['A', 'B']

### Problem 2: Directed Graph

![Example Graph 2](https://upload.wikimedia.org/wikipedia/commons/5/51/Directed_graph.svg)

Encode a direct graph and find the route between 1 and 4.

In [ ]:
grph2 = DiGraph()

In [20]:
#Encode your graph here

In [ ]:
origin = 1
destination = #your code here

In [ ]:
parents = bfs(graph = grph2,
              origin = origin,
              destination = destination)

getRoute(parents, destination)

Problem 2.1:

Using BFS, how can you use the parents dictionary to tell if you can create the destination node from the origin node. 

Example: Can you reach node 4 from node 2? How can you tell from parents variable

#### Extra Reading

DFS and BFS algorithms work on unweighted graphs. In the example of the flight chart, BFS returns the route with the lowest amount of edges (flight changes). This ignores the actual distance between the 2 points (example: BFS from La Jolla to Toronto).

If you are wondering how to find the path with the shortest distance traveled, you should look towards some weighted graph algorithms. Here are some suggestions on further readings if you are interested:

    - Djikstra: https://medium.com/basecs/finding-the-shortest-path-with-a-little-help-from-dijkstra-613149fbdc8e
    - A*: https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2